In [55]:
import requests
import re

import pandas as pd

In [56]:
drug_prefix = "https://rest.kegg.jp/get/"

In [57]:
rx_MOL_WEIGHT = re.compile(r"MOL_WEIGHT\s+(\S+)")
rx_EFFICACY = re.compile(r"EFFICACY\s+(.+)")
rx_TARGET = re.compile(r"(K\d+)")

In [58]:
content = ",".join(["drug_id", "name", "mol_weight", "efficacy"]) + "\n"

df = pd.read_csv("parsed_file/drug_temp.csv")

drug_kegg = {}

for index, row in df.iterrows():


    mol_weight = 0
    efficacy = ""
    

    
    is_target = False


    drug_id = row['ko']
    drug_name = row['name']

    drug_kegg[drug_id] = set()

    drug_url = drug_prefix + drug_id
    drug_file = requests.get(drug_url).text
    
    for l in drug_file.split("\n"):
        if not l.startswith(" "):
            is_target = False


        if l.startswith("MOL_WEIGHT"):
            search_weight = rx_MOL_WEIGHT.search(l)

            if search_weight:
                mol_weight = float(search_weight.group(1))
            
        elif l.startswith("EFFICACY"):
            search_efficacy = rx_EFFICACY.search(l)

            if search_efficacy:
                efficacy = search_efficacy.group(1)
        
        elif l.startswith("TARGET"):
            is_target = True
        
        
        
        if is_target == True:
            targets = rx_TARGET.findall(l)

            for t in targets:
                drug_kegg[drug_id].add(t)
    
    content += f'"{drug_id}","{drug_name}","{mol_weight}","{efficacy}"\n'


In [59]:
with open("import/drug.csv", "w") as output:
    output.write(content)

In [31]:
with open("import/drug_kegg.csv", "w") as output:
    output.write("drug_id,kegg_id,")

with open("import/drug_kegg.csv", "a") as output:
    for drug in drug_kegg:
        for k in drug_kegg[drug]:
            output.write(",".join([drug, k]) + "\n")

In [47]:
df_2 = pd.read_csv("import/kegg.csv")

keggs = set(df_2.ko.unique())


keggs



{'K00985',
 'K01143',
 'K01516',
 'K05345',
 'K07132',
 'K12969',
 'K12970',
 'K18942',
 'K18943',
 'K18944',
 'K18945',
 'K18946',
 'K18947',
 'K18948',
 'K18949',
 'K18950',
 'K18951',
 'K18959',
 'K18960',
 'K18961',
 'K18962',
 'K18963',
 'K18964',
 'K18969',
 'K19247',
 'K19248',
 'K19249',
 'K19250',
 'K19251',
 'K19252',
 'K19253',
 'K19254',
 'K19255',
 'K19256',
 'K19257',
 'K19258',
 'K19259',
 'K19260',
 'K19261',
 'K19262',
 'K19263',
 'K19264',
 'K19287',
 'K19288',
 'K19358',
 'K19376',
 'K19377',
 'K19378',
 'K19379',
 'K19380',
 'K19381',
 'K19388',
 'K19389',
 'K19390',
 'K19391',
 'K19392',
 'K19393',
 'K19394',
 'K19395',
 'K19396',
 'K19397',
 'K19434',
 'K19435',
 'K19436',
 'K19437',
 'K19438',
 'K19439',
 'K19440',
 'K19441',
 'K19442',
 'K19443',
 'K19444',
 'K19445',
 'K19446',
 'K19447',
 'K19448',
 'K19450',
 'K19451',
 'K19452',
 'K19453',
 'K19454',
 'K19455',
 'K19456',
 'K19457',
 'K19458',
 'K19459',
 'K19460',
 'K19461',
 'K19462',
 'K19463',
 'K19796',

In [48]:

rx_name = re.compile(r'NAME\s+(.+)')

In [49]:
new_kegg = set()

for drug in drug_kegg:
    for k in drug_kegg[drug]:
        if k not in keggs:
            new_kegg.add(k)

print (new_kegg)

{'K05208', 'K05210', 'K08524', 'K00560', 'K05209', 'K05404', 'K11787', 'K08527', 'K05130', 'K05131', 'K03164', 'K08526', 'K08529', 'K08525', 'K17689', 'K05771', 'K05212', 'K06744', 'K00287', 'K05211', 'K07375', 'K17692', 'K06454', 'K17691', 'K17690', 'K05080', 'K08528', 'K04180', 'K06745'}


In [52]:
content = ""

for k in new_kegg:
    kegg_url = drug_prefix + k
    #print (kegg_url)
    kegg_page = requests.get(kegg_url).text
    

    for line in kegg_page.split("\n"):
        if line.startswith("NAME"):
            search_name = rx_name.search(line)
            #print (line)
            if search_name:
                name = search_name.group(1)

                content += f'"{k}","{name}"\n'



In [53]:
content

'"K05208","glutamate receptor ionotropic, NMDA 1"\n"K05210","glutamate receptor ionotropic, NMDA 2B"\n"K08524","retinoid X receptor alpha"\n"K00560","thymidylate synthase [EC:2.1.1.45]"\n"K05209","glutamate receptor ionotropic, NMDA 2A"\n"K05404","toll-like receptor 7"\n"K11787","phosphoribosylamine--glycine ligase / phosphoribosylglycinamide formyltransferase / phosphoribosylformylglycinamidine cyclo-ligase [EC:6.3.4.13 2.1.2.2 6.3.3.1]"\n"K08527","retinoic acid receptor alpha"\n"K05130","interferon receptor 1"\n"K05131","interferon receptor 2"\n"K03164","DNA topoisomerase II [EC:5.6.2.2]"\n"K08526","retinoid X receptor gamma"\n"K08529","retinoic acid receptor gamma"\n"K08525","retinoid X receptor beta"\n"K17689","cytochrome P450 family 3 subfamily A4 [EC:1.14.13.32 1.14.14.55 1.14.14.56 1.14.14.57 1.14.14.73 1.14.14.-]"\n"K05771","glucocorticoid receptor"\n"K05212","glutamate receptor ionotropic, NMDA 2D"\n"K06744","programmed cell death protein 1"\n"K00287","dihydrofolate reductase 

In [54]:
with open("import/kegg.csv", "a") as output:
    output.write("\n")
    output.write(content)